In [ ]:
import numpy
import xarray
import matplotlib.pyplot as mp
import glob
import pandas
import cartopy

In [ ]:
#PRECT_root = '/ninad/NCAR_LENS/PRECT/'
PRECT_root = '/DFS-L/DATA/pritchard/blangenb/NCAR_LENS/PRECT/'

In [ ]:
lon_qtr = int(144/4)
lat_qtr = int(192/4)
lon_half = int(144/2)
lat_half = int(192/2)

Open pre-industrial control as multi-file dataset

In [ ]:
# prect_pic_mfds = xarray.open_mfdataset(PRECT_root + 'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.*.nc', \
#                                       chunks={'lat':12, 'lon':12}).isel(lat=slice(0,lat_qtr), lon=slice(0,lon_qtr))
prect_pic_mfds = xarray.open_mfdataset(PRECT_root + 'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.*.nc', \
                                       chunks={'lat':12, 'lon':12}).isel(lat=slice(0,lat_half), lon=slice(0,lon_half))
# prect_pic_mfds = xarray.open_mfdataset(PRECT_root + 'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.*.nc').isel(lat=slice(0,5), lon=slice(0,5))

In [ ]:
prect_pic = prect_pic_mfds['PRECT']*86400.*1000.
lon = prect_pic_mfds['lon']
lat = prect_pic_mfds['lat']
time = prect_pic_mfds['time'].values

for checking memory usage

get dates corresponding to ONDJFM

In [ ]:
time_datetime = numpy.copy(time) #pandas.to_datetime(time)
time_ondjfm_indices = [d.month in [10,11,12,1,2,3] for d in time_datetime]

In [ ]:
prect_pic_seas = prect_pic.isel(time=time_ondjfm_indices)

In [ ]:
prect_pic_seas

In [ ]:
prect_pic_seas_data = prect_pic_seas.values

In [ ]:
prect_pic_seas_data.shape

set all values less than 0.1 to NaN

In [ ]:
prect_rcp_seas

In [ ]:
%time
#prect_rcp = prect_rcp.where(prect_rcp<0.1, drop=True).values
prect_pic_seas_filtered = prect_pic_seas.where(prect_pic_seas>0.1, drop=True)#.values
#prect_rcp = prect_rcp.values
#prect_rcp[prect_rcp<0.1]=numpy.nan

In [ ]:
prect_pic_timemean = numpy.nanmean(prect_rcp, axis=0)
prect_ondjfm_timemean = numpy.nanmean(prect_rcp[time_ondjfm_indices,:,:], axis=0)

In [ ]:
prect_djf = prect_rcp[time_ondjfm_indices,:,:]

In [ ]:
prect_djf.shape

In [ ]:
proj_map = cartopy.crs.PlateCarree()

fig = mp.figure(figsize=(10,10))

ax = fig.add_subplot(111, projection=proj_map)
ax.coastlines(resolution='50m')
plot = ax.contourf(prect_pic_mfds['lon'].values, prect_pic_mfds['lat'].values, prect_pic_timemean)
mp.colorbar(plot)
#ax.set_extent([-130,-100,20,55])
ax.set_extent([-150,-80,10,70])

In [ ]:
ndays,nlat,nlon = prect_rcp.shape

* do 100 year precipitation days

In [ ]:
prect_djf.shape

In [ ]:
time_datetime_djf = time_datetime[time_ondjfm_indices]

In [ ]:
nyears = time_datetime_djf[-1].year-time_datetime_djf[0].year+1

In [ ]:
# to get a 100-year event, you want to figure out how many years are involved
# a 100-year event would happen 18 times in nyears

In [ ]:
return_period = 200 # in years
#events_per_year = 151/40
events_per_year = 151

#return_period = 100
#events_per_year = 1

return_val_perc = 100*(1-1/(return_period*events_per_year))
print(return_val_perc)

In [ ]:
percentile_values = numpy.zeros((nlat,nlon))
for j in range(nlat):
    for k in range(nlon):
        # get rid of nans
        tmp_distro = prect_djf[:,j,k]
        #prect_djf[:,j,k][~numpy.isnan(prect_djf[:,j,k])]
        percentile_values[j,k] = numpy.nanpercentile(tmp_distro, return_val_perc)

In [ ]:
proj_map = cartopy.crs.PlateCarree()

fig = mp.figure(figsize=(10,10))

levels = numpy.arange(0,201,20)
ax = fig.add_subplot(111, projection=proj_map)
ax.coastlines(resolution='50m')
plot = ax.contourf(prect_pic_mfds['lon'].values, prect_pic_mfds['lat'].values, percentile_values, levels=levels)
mp.colorbar(plot)
#ax.set_extent([-130,-100,20,55])
ax.set_extent([-130,-100,30,55])